In [1]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
n_classes = 10
batch_size = 64
input_shape = x_train.shape[1] * x_train.shape[1]

In [4]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train = x_train.reshape(x_train.shape[0], input_shape)
x_test = x_test.reshape(x_test.shape[0], input_shape)

y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)

In [5]:
input = Input(shape=(input_shape,))

x = Dense(512, activation='relu')(input)
x = Dense(64, activation='relu')(x)
output = Dense(n_classes, activation='softmax')(x)

model = Model(inputs=input, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 64)                32832     
                                                                 
 dense_2 (Dense)             (None, 10)                650       
                                                                 
Total params: 435,402
Trainable params: 435,402
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer="adam", metrics=["accuracy"], loss="categorical_crossentropy")

In [7]:
callbacks = [
    EarlyStopping(patience=5, monitor='val_accuracy', mode="auto"),
    ReduceLROnPlateau(factor=0.1, patience=2, min_lr=0.0001, monitor='val_accuracy', mode='max', verbose=1),
    ModelCheckpoint('model.h5', save_best_only=True, save_weights_only=True, monitor='val_accuracy', mode='max', verbose=1)
]

In [8]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=30, validation_split=0.1, callbacks=callbacks)

Epoch 1/30
839/844 [============================>.] - ETA: 0s - loss: 0.2259 - accuracy: 0.9335
Epoch 1: val_accuracy improved from -inf to 0.97083, saving model to model.h5
844/844 [==============================] - 8s 9ms/step - loss: 0.2255 - accuracy: 0.9336 - val_loss: 0.1007 - val_accuracy: 0.9708 - lr: 0.0010
Epoch 2/30
843/844 [============================>.] - ETA: 0s - loss: 0.0847 - accuracy: 0.9746
Epoch 2: val_accuracy improved from 0.97083 to 0.97383, saving model to model.h5
844/844 [==============================] - 7s 9ms/step - loss: 0.0847 - accuracy: 0.9746 - val_loss: 0.0844 - val_accuracy: 0.9738 - lr: 0.0010
Epoch 3/30
840/844 [============================>.] - ETA: 0s - loss: 0.0571 - accuracy: 0.9818
Epoch 3: val_accuracy improved from 0.97383 to 0.97883, saving model to model.h5
844/844 [==============================] - 7s 8ms/step - loss: 0.0571 - accuracy: 0.9818 - val_loss: 0.0777 - val_accuracy: 0.9788 - lr: 0.0010
Epoch 4/30
837/844 [====================

In [9]:
model.load_weights("model.h5")

In [10]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.05854372680187225
Test accuracy: 0.9846000075340271
